In [ ]:
import requests
from geopy.distance import geodesic

# 1. Find weather forecast
def find_weather_forecast(date, location):
    api_key = 'your_openweather_api_key'
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={location}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    
    # Assuming we get a forecast and we parse it based on the date
    for forecast in data['list']:
        if forecast['dt_txt'].startswith(date):
            return forecast['weather'][0]['description'], forecast['main']['temp']
    return "No forecast found for the given date."

# 2. Rank appropriate locations based on user input using a RAG system
def rank_appropriate_locations(user_prompt, chroma_client):
    # Extract preferences from user prompt
    preferences = extract_preferences(user_prompt)  # e.g., sun, beach, history
    
    # Query ChromaDB based on extracted preferences
    relevant_docs = query_chromadb(preferences, chroma_client)
    
    # Score locations based on relevance
    ranked_locations = score_locations(relevant_docs, preferences)
    
    # Return the ranked list of locations
    return ranked_locations
# 3. Find information on a location
def find_info_on_location(user_prompt, location):
    # Use RAG to retrieve documents or information related to the location
    documents = search_documents_for_location(location)  # Placeholder for your RAG search
    relevant_info = extract_relevant_info(documents, user_prompt)  # Another placeholder
    return relevant_info

# 4. Add location to visit list
def add_location_to_visit_list(visit_list, location):
    visit_list.append(location)
    return visit_list

# 5. Compute distance to locations in list
def compute_distance_to_list(visit_list, new_location):
    new_location_coords = get_coordinates(new_location)  # Placeholder for geocoding
    min_distance = float('inf')
    
    for loc in visit_list:
        loc_coords = get_coordinates(loc)  # Geocoding for each location
        distance = geodesic(loc_coords, new_location_coords).km
        if distance < min_distance:
            min_distance = distance
    
    return min_distance

# Helper function (you'll need geocoding services like OpenCage or Google Maps)
def get_coordinates(location):
    # Placeholder function to return lat/lng for a location
    # You can use OpenCage API, Google Maps API, etc. to implement this
    pass


In [ ]:
# used in get_appropriate_locations

from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def extract_preferences(prompt):
    # Define potential location preferences
    candidate_labels = ["beach", "historical", "mountains", "nightlife", "nature", "museum", "sunny", "cold climate", "shopping", "sports"]
    
    # Use zero-shot classification to predict the user's preferences
    classification = classifier(prompt, candidate_labels)
    
    # Extract the most relevant preferences based on the classification scores
    preferences = [label for label, score in zip(classification['labels'], classification['scores']) if score > 0.5]
    
    return preferences

def query_chromadb(preferences, chroma_client):
    # Query ChromaDB using the preferences
    results = []
    for preference in preferences:
        results.extend(chroma_client.query(preference))  # Use similarity search
    return results

def score_locations(docs, preferences):
    location_scores = {}
    for doc in docs:
        location = doc['location']
        score = sum(1 for pref in preferences if pref in doc['description'])
        location_scores[location] = location_scores.get(location, 0) + score
    # Sort locations by score
    ranked_locations = sorted(location_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_locations

In [ ]:
# get user intent
def get_intent(prompt):
    """
    Very basic intent recognition based on user input.
    This can be replaced with more sophisticated NLP-based classification.
    """
    if "weather" in prompt.lower():
        return "weather_forecast"
    elif "visit" in prompt.lower() or "places" in prompt.lower():
        return "location_recommendation"
    elif "information" in prompt.lower() or "tell me about" in prompt.lower():
        return "location_info"
    else:
        return "general_chat"

# Example usage
# user_prompt = "What are the best sunny places to visit?"
# intent = get_intent(user_prompt)
# print(intent)  # Output: location_recommendation


In [ ]:
def chatbot(user_prompt, visit_list):
    intent = get_intent(user_prompt)

    if intent == "weather_forecast":
        # Extract the date and location from the prompt
        return find_weather_forecast("tomorrow", "San Juan")

    elif intent == "location_recommendation":
        # Use the rank_appropriate_locations function
        return rank_appropriate_locations(user_prompt)

    elif intent == "location_info":
        # Extract location and find info
        return find_info_on_location(user_prompt, "Old San Juan")

    elif intent == "add_location":
        # Add a location to visit list
        return add_location_to_visit_list(visit_list, "San Juan Beach")

    else:
        return "I'm not sure how to help with that. Can you ask me about the weather or places to visit?"
